# Data Cleaning Project - Cleaning 3M+ Rows of data in pandas

In [17]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv('h1b_kaggle.csv')
df.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.0,2016.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674.0,2016.0,"PLANO, TEXAS",-96.698886,33.019843
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066.0,2016.0,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314.0,2016.0,"DENVER, COLORADO",-104.990251,39.739236
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518.4,2016.0,"ST. LOUIS, MISSOURI",-90.199404,38.627003


In [19]:
initial_shape = df.shape

initial_nulls = df.isnull().sum()
initial_duplicates = df.duplicated().sum()
print('----------Initial data set overview------------')
print("Initial Shape:", initial_shape)
print("Initial Duplicates:", initial_duplicates)
print("Initial Missing Values:\n", initial_nulls.sort_values(ascending=False).head(10))


----------Initial data set overview------------
Initial Shape: (3002458, 11)
Initial Duplicates: 0
Initial Missing Values:
 lon                   107242
lat                   107242
SOC_NAME               17734
PREVAILING_WAGE           85
EMPLOYER_NAME             59
JOB_TITLE                 43
FULL_TIME_POSITION        15
YEAR                      13
CASE_STATUS               13
Unnamed: 0                 0
dtype: int64


In [20]:
# Cleaning column Names
df.columns = (df.columns.str.strip().str.title().str.replace(" ", "_"))

In [21]:
# Check for duplicates
initial_rows = df.shape[0]
duplicates = df.duplicated().sum()

print(f"Number of duplicate rows found: {duplicates}")

if duplicates > 0:
    df.drop_duplicates(inplace=True)
    print(f"Removed {duplicates} duplicate rows.")
else:
    print("No duplicates found.")

print(f"Current row count: {df.shape[0]}")

Number of duplicate rows found: 0
No duplicates found.
Current row count: 3002458


In [22]:
df.drop(columns='Unnamed:_0',inplace=True)
print('Dropped unnamed:_0 column')

Dropped unnamed:_0 column


In [23]:
# List of columns where missing data is negligible (<0.01%)
trivial_cols = ['Case_Status', 'Year', 'Full_Time_Position']

# Droping  rows where these specific columns are null
df.dropna(subset=trivial_cols, inplace=True)

print("Dropped rows with missing Case Status, Year, or Position.")
print(f"New dataset shape: {df.shape}")

Dropped rows with missing Case Status, Year, or Position.
New dataset shape: (3002443, 10)


In [24]:
# Imputing text columns with "Unknown" to avoid hallucinating data
text_cols = ['Employer_Name', 'Soc_Name', 'Job_Title']
for col in text_cols:
    df[col] = df[col].fillna('Unknown')

print("Filled missing values from Employer_Name , Soc_Name and Job_Title columns with 'Unknown'.")

Filled missing values from Employer_Name , Soc_Name and Job_Title columns with 'Unknown'.


In [25]:
# Impute Prevailing Wage using the Median of the specific SOC_NAME (Job Category)
df['Prevailing_Wage'] = df['Prevailing_Wage'].fillna(df.groupby('Soc_Name')['Prevailing_Wage'].transform('median'))

print("Imputed missing wages using median wage per Job Category (SOC_NAME).")

Imputed missing wages using median wage per Job Category (SOC_NAME).


In [26]:
# Impute missing Lat/Lon based on the WORKSITE
# If we know the city (Worksite), we can copy the lat or lon from other rows with the same city
df['Lon'] = df['Lon'].fillna(df.groupby('Worksite')['Lon'].transform('mean'))
df['Lat'] = df['Lat'].fillna(df.groupby('Worksite')['Lat'].transform('mean'))

df.dropna(subset=['Lon', 'Lat'], inplace=True)

print("Imputed coordinates based on Worksite location matching.")

Imputed coordinates based on Worksite location matching.


In [27]:
df.dropna(inplace=True)

In [29]:
# Visualize wage distribution to spot outliers 
print(df['Prevailing_Wage'].describe())

Q1 = df['Prevailing_Wage'].quantile(0.25)
Q3 = df['Prevailing_Wage'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = 15000  
upper_bound = Q3 + 3 * IQR 


count    2.895201e+06
mean     1.452208e+05
std      5.299251e+06
min      0.000000e+00
25%      5.462100e+04
50%      6.512500e+04
75%      8.151500e+04
max      6.997607e+09
Name: Prevailing_Wage, dtype: float64


In [30]:
outliers_low = df[df['Prevailing_Wage'] < lower_bound].shape[0]
df = df[df['Prevailing_Wage'] >= lower_bound]

print(f"Removed {outliers_low} rows with unrealistic wages < ${lower_bound}")

# Optional: Cap the super high wages instead of dropping them
# This keeps the data but stops one typo from ruining the average
def cap_wages(wage):
    if wage > upper_bound:
        return upper_bound
    return wage

df['Prevailing_Wage'] = df['Prevailing_Wage'].apply(cap_wages)
print(f"Capped wages above ${upper_bound:,.2f} to reduce skew.")


Removed 78 rows with unrealistic wages < $15000
Capped wages above $162,197.00 to reduce skew.


In [31]:
# Preview clean data
df.head()

,Case_Status,Employer_Name,Soc_Name,Job_Title,Full_Time_Position,Prevailing_Wage,Year,Worksite,Lon,Lat
0,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.0,2016.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,162197.0,2016.0,"PLANO, TEXAS",-96.698886,33.019843
2,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,162197.0,2016.0,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,162197.0,2016.0,"DENVER, COLORADO",-104.990251,39.739236
4,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518.4,2016.0,"ST. LOUIS, MISSOURI",-90.199404,38.627003


In [34]:
Final_shape = df.shape

Final_nulls = df.isnull().sum()
Final_duplicates = df.duplicated().sum()
print('----------Final Cleaned data set overview------------')
print("Final Shape:", Final_shape)
print("Final Duplicates:", Final_duplicates)
print("Final Missing Values:\n", Final_nulls.sort_values(ascending=False).head(10))


----------Final Cleaned data set overview------------
Final Shape: (2895123, 10)
Final Duplicates: 924030
Final Missing Values:
 Case_Status           0
Employer_Name         0
Soc_Name              0
Job_Title             0
Full_Time_Position    0
Prevailing_Wage       0
Year                  0
Worksite              0
Lon                   0
Lat                   0
dtype: int64
